<a href="https://colab.research.google.com/github/liliya111darky/logical-ai/blob/notebooks/04_full_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 16.0 MB/s eta 0:00:00


In [5]:
!pip install transformers

In [6]:
# ---------- Импорт библиотек ----------
import sympy as sp
from sympy import sympify
from z3 import *
import sqlite3
from transformers import pipeline

# NLP-модель
nlp_model = pipeline("sentiment-analysis")

# ---------- Память ----------
DB_PATH = "facts.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS facts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            text TEXT UNIQUE,
            type TEXT,
            extra TEXT
        )
    """)
    conn.commit()
    conn.close()

def save_fact(text, fact_type, extra_info=""):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    try:
        cur.execute("INSERT INTO facts (text, type, extra) VALUES (?, ?, ?)",
                    (text, fact_type, str(extra_info)))
        conn.commit()
    except sqlite3.IntegrityError:
        pass
    conn.close()

def get_all_facts():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT text, type, extra FROM facts")
    rows = cur.fetchall()
    conn.close()
    return rows

def contradicts_existing(new_text):
    all_facts = [row[0] for row in get_all_facts()]
    neg_versions = [f"не ({new_text})", f"НЕ {new_text}", f"not ({new_text})"]
    return any(fact in neg_versions for fact in all_facts)

init_db()

# ---------- Математическая проверка ----------
def check_math(statement):
    try:
        if "=" in statement:
            left, right = statement.split("=")
            return sp.simplify(sympify(left) - sympify(right)) == 0
        else:
            return bool(sympify(statement))
    except:
        return None

# ---------- Логическая проверка через Z3 ----------
def check_implication(conditions, conclusion):
    solver = Solver()
    A, B, C, D = Bools('A B C D')  # до 4 переменных
    cond_expr = eval(conditions)
    concl_expr = eval(conclusion)
    solver.add(cond_expr, Not(concl_expr))
    return solver.check() == unsat

# ---------- Единый анализ ----------
def analyze_input(text):
    # 1. Проверка на противоречие
    if contradicts_existing(text):
        return {"type": "conflict", "message": "⚠️ Противоречие с памятью"}

    # 2. Математика
    if any(op in text for op in ["=", ">", "<"]):
        math_result = check_math(text)
        if math_result is True:
            save_fact(text, "math_true")
            return {"type": "math_true", "message": "✅ Математический факт сохранён"}
        elif math_result is False:
            return {"type": "math_false", "message": "❌ Математическая проверка не пройдена"}

    # 3. Логика "если ... то ..."
    if "если" in text.lower() and "то" in text.lower():
        try:
            parts = text.lower().replace("если", "").split("то")
            conds = parts[0].strip()
            concl = parts[1].strip()
            if check_implication(conds, concl):
                save_fact(text, "valid_implication")
                return {"type": "valid_implication", "message": "✅ Логическая импликация сохранена"}
            else:
                return {"type": "invalid_implication", "message": "❌ Импликация неверна"}
        except:
            pass

    # 4. NLP-анализ текста
    sentiment = nlp_model(text)[0]
    return {"type": "text", "sentiment": sentiment}

# ---------- Примеры ----------
print(analyze_input("2+2=4"))
print(analyze_input("Если A > B и B > C то A > C"))
print(analyze_input("Сегодня солнечно и тепло"))
print("Память:", get_all_facts())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


{'type': 'math_true', 'message': '✅ Математический факт сохранён'}
{'type': 'text', 'sentiment': {'label': 'NEGATIVE', 'score': 0.6741251945495605}}
{'type': 'text', 'sentiment': {'label': 'POSITIVE', 'score': 0.7894664406776428}}
Память: [('2+2=4', 'math_true', '')]
